In [ ]:
import ray
ray.init(num_cpus=20) # In theory auto sensed, in practice... eh

In [ ]:
import sys
import time
import timeit
import threading

In [ ]:

#tag::variable_sleep_task[]
@ray.remote
def remote_task(x):
    time.sleep(x)
    return x
#end::variable_sleep_task[]

things = list(range(10))
things.sort(reverse=True)

In [ ]:
#tag::get_only[]
# Process in order
def in_order():
    # Make the futures
    futures = list(map(lambda x: remote_task.remote(x), things))
    values = ray.get(futures)
    for v in values:
        print(f" Completed {v}")
        time.sleep(1) # Business logic goes here
#tag::get_only[]

In [ ]:
# Currently ray.wait will not return more than num_returns objects as ready
# even if there are. This might change so even when fetching an object at a time
# iterate over the result.
#tag::as_available[]
# Process as results become available
def as_available():
    # Make the futures
    futures = list(map(lambda x: remote_task.remote(x), things))
    # While we still have pending futures
    while len(futures) > 0:
        ready_futures, rest_futures = ray.wait(futures)
        print(f"Ready {len(ready_futures)} rest {len(rest_futures)}")
        for id in ready_futures:
            print(f'completed value {id}, result {ray.get(id)}')
            time.sleep(1) # Business logic goes here
        # We just need to wait on the ones which are not yet available
        futures = rest_futures
#end::as_available[]

In [ ]:
timeit.timeit(lambda: as_available(), number=1)

In [ ]:
timeit.timeit(lambda: in_order(), number=1)

In [ ]:
#tag::handle_bad_futures[]
futures = list(map(lambda x: remote_task.remote(x), [1, threading.TIMEOUT_MAX]))
# While we still have pending futures
while len(futures) > 0:
    # In practice 10 seconds is too short for most cases.
    ready_futures, rest_futures = ray.wait(futures, timeout=10, num_returns=1)
    # If we get back anything less than num_returns 
    if len(ready_futures) < 1:
        print(f"Timed out on {rest_futures}")
        # You don't _have to cancel_ but if you've your task is using a lot of resources
        ray.cancel(*rest_futures)
        # You should break since you exceeded your timeout
        break
    for id in ready_futures:
        print(f'completed value {id}, result {ray.get(id)}')
        futures = rest_futures
#end::handle_bad_futures[]